In [4]:
from tile_classifier import TileClassifier
from letter_classifier import LetterClassifier

from keras import backend as K
import tensorflow as tf
from tensorflow.python.tools import freeze_graph
from tensorflow.python.tools import optimize_for_inference_lib

def export_model(saver, model, input_node_names, output_node_name,MODEL_NAME):
    tf.train.write_graph(K.get_session().graph_def, 'out', \
        MODEL_NAME + '_graph.pbtxt')

    saver.save(K.get_session(), 'out/' + MODEL_NAME + '.chkp')

    freeze_graph.freeze_graph('out/' + MODEL_NAME + '_graph.pbtxt', None, \
        False, 'out/' + MODEL_NAME + '.chkp', output_node_name, \
        "save/restore_all", "save/Const:0", \
        'out/frozen_' + MODEL_NAME + '.pb', True, "")

    input_graph_def = tf.GraphDef()
    with tf.gfile.Open('out/frozen_' + MODEL_NAME + '.pb', "rb") as f:
        input_graph_def.ParseFromString(f.read())

    output_graph_def = optimize_for_inference_lib.optimize_for_inference(
            input_graph_def, input_node_names, [output_node_name],
            tf.float32.as_datatype_enum)

    with tf.gfile.FastGFile('out/opt_' + MODEL_NAME + '.pb', "wb") as f:
        f.write(output_graph_def.SerializeToString())

    print("graph saved!")

In [7]:
if __name__ == '__main__':
    # model = TileClassifier('./data/data_set_tile','tile_classifier.h5')
    model = LetterClassifier('./data/data_set_letter','letter_classifier.h5')
    model.train_evaluate(epochs=25)
    class_indices = model.get_class_indices()
    print(class_indices)
    

Found 812 images belonging to 33 classes.
Epoch 1/25
649/649 [==============================] - 11s 17ms/step - loss: 3.3933 - acc: 0.0801
Epoch 2/25
649/649 [==============================] - 13s 20ms/step - loss: 3.3180 - acc: 0.0724
Epoch 3/25
649/649 [==============================] - 12s 19ms/step - loss: 3.3032 - acc: 0.0894
Epoch 4/25
649/649 [==============================] - 12s 19ms/step - loss: 3.2866 - acc: 0.1002
Epoch 5/25
649/649 [==============================] - 15s 23ms/step - loss: 3.1824 - acc: 0.1464
Epoch 6/25
649/649 [==============================] - 14s 22ms/step - loss: 2.5481 - acc: 0.3112
Epoch 7/25
649/649 [==============================] - 15s 23ms/step - loss: 1.5122 - acc: 0.5701
Epoch 8/25
649/649 [==============================] - 17s 26ms/step - loss: 0.7611 - acc: 0.7920
Epoch 9/25
649/649 [==============================] - 13s 21ms/step - loss: 0.3990 - acc: 0.8690
Epoch 10/25
649/649 [==============================] - 12s 18ms/step - loss: 0.2160 -

In [8]:
export_model(tf.train.Saver(), model, ["conv2d_1_input"], "dense_2/Softmax", "letter_classifier")

INFO:tensorflow:Restoring parameters from out/letter_classifier.chkp
INFO:tensorflow:Froze 16 variables.
Converted 16 variables to const ops.
graph saved!
